## Bias Mitigation using AIF360 - Heart Failure Prediction Dataset (Source: https://www.kaggle.com/datasets/fedesoriano/heart-failure-prediction/data)
Model Training and Evaluation

In [23]:
#load preprocessed data 
import pandas as pd
train_df = pd.read_csv("./data_subsets/train_75M_25F.csv")

X_test = pd.read_csv("./data_splits/X_test.csv")
y_test = pd.read_csv("./data_splits/y_test.csv")

#check out the data
train_df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,61,1,3,146.0,241.0,0,0,148.0,1,3.0,0,1
1,52,1,1,120.0,284.0,0,0,118.0,0,0.0,2,0
2,48,0,3,150.0,227.0,0,0,130.0,1,1.0,1,0
3,49,1,3,128.0,212.0,0,0,96.0,1,0.0,1,1
4,56,1,3,120.0,236.0,0,1,148.0,0,0.0,1,1


In [24]:
TARGET = "HeartDisease"
SENSITIVE = "Sex"   # 1 = Male, 0 = Female

categorical_cols = ['Sex','ChestPainType','FastingBS','RestingECG','ExerciseAngina','ST_Slope']
continuous_cols  = ['Age','RestingBP','Cholesterol','MaxHR','Oldpeak']

In [25]:
# Split train into X / y and keep sensitive feature for fairness evaluation
X_train = train_df.drop(columns=[TARGET])
y_train = train_df[TARGET]

In [26]:
# scale numeric features only, fit on train, transform test
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_num_scaled = pd.DataFrame(
    scaler.fit_transform(X_train[continuous_cols]),
    columns=continuous_cols, index=X_train.index
)
X_test_num_scaled = pd.DataFrame(
    scaler.transform(X_test[continuous_cols]),
    columns=continuous_cols, index=X_test.index
)

In [27]:
#one-hot encode categoricals; numeric are kept as is 
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(handle_unknown="ignore", drop="if_binary", sparse_output=False)
ohe.fit(X_train[categorical_cols])

X_train_cat = pd.DataFrame(
    ohe.transform(X_train[categorical_cols]),
    columns=ohe.get_feature_names_out(categorical_cols),
    index=X_train.index
)
X_test_cat = pd.DataFrame(
    ohe.transform(X_test[categorical_cols]),
    columns=ohe.get_feature_names_out(categorical_cols),
    index=X_test.index
)

In [28]:
# Assemble final matrices
X_train_ready = pd.concat([X_train_cat, X_train_num_scaled], axis=1)
X_test_ready  = pd.concat([X_test_cat,  X_test_num_scaled],  axis=1)

print("Final feature shapes:", X_train_ready.shape, X_test_ready.shape)

Final feature shapes: (600, 18) (184, 18)


In [29]:
# create sensitive attribute arrays - after creating X_train_ready and X_test_ready
A_train = X_train["Sex"].astype(int).to_numpy().ravel()  # 1=Male, 0=Female
A_test  = X_test["Sex"].astype(int).to_numpy().ravel()

### Traditional ML Models - Baseline: K-Nearest Neighbors (KNN) & Decision Tree (DT)

In [30]:
#import required libraries
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix
)

#define a function 
def evaluate_model(y_true, y_pred, model_name):
    print(f"=== {model_name} Evaluation ===")
    print("Accuracy :", accuracy_score(y_true, y_pred))
    print("Precision:", precision_score(y_true, y_pred, average='binary'))
    print("Recall   :", recall_score(y_true, y_pred, average='binary'))
    print("F1 Score :", f1_score(y_true, y_pred, average='binary'))
    print("\nClassification Report:\n", classification_report(y_true, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))
    print("\n" + "="*40 + "\n")

### PCA-KNN

In [31]:
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix
)
import numpy as np

#1) PCA + KNN pipeline (on one-hot encoded + scaled features)
pca_knn = Pipeline([
    ('pca', PCA(n_components=0.95, random_state=42)),  # keep 95% variance
    ('knn', KNeighborsClassifier(
        n_neighbors=15, metric='manhattan', weights='distance'
    ))
])

pca_knn.fit(X_train_ready, y_train)

# Inspect PCA details
n_comp = pca_knn.named_steps['pca'].n_components_
expl_var = pca_knn.named_steps['pca'].explained_variance_ratio_.sum()
print(f"PCA components: {n_comp} | Explained variance retained: {expl_var:.3f}")

# 2) Evaluate 
y_pred_pca_knn = pca_knn.predict(X_test_ready)
probs_pca_knn = pca_knn.predict_proba(X_test_ready)[:, 1]
  
evaluate_model(y_test, y_pred_pca_knn, "KNN (best params)")

PCA components: 12 | Explained variance retained: 0.967
=== KNN (best params) Evaluation ===
Accuracy : 0.8858695652173914
Precision: 0.9090909090909091
Recall   : 0.8823529411764706
F1 Score : 0.8955223880597015

Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.89      0.87        82
           1       0.91      0.88      0.90       102

    accuracy                           0.89       184
   macro avg       0.88      0.89      0.88       184
weighted avg       0.89      0.89      0.89       184

Confusion Matrix:
 [[73  9]
 [12 90]]




### Tuned Decision Tree (DT)

In [32]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import (
    classification_report, confusion_matrix, accuracy_score,
    precision_score, recall_score, f1_score
)

# 1) Base model
dt = DecisionTreeClassifier(random_state=42)

# 2) Hyperparameter grid 
param_grid = {
    "criterion": ["gini", "entropy"],
    "max_depth": [3, 5, 7, 9, None],
    "min_samples_split": [2, 5, 10, 20],
    "min_samples_leaf": [1, 2, 4, 6, 10],
}

# 3) Cross-validation setup
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 4) Grid search 
grid_dt = GridSearchCV(
    estimator=dt,
    param_grid=param_grid,
    cv=cv,
    scoring="f1",      
    n_jobs=-1,
    verbose=0
)

grid_dt.fit(X_train_ready, y_train)

print("Best Decision Tree params:", grid_dt.best_params_)
print("Best CV F1:", grid_dt.best_score_)

# 5) Train & evaluate best DT
tuned_dt = grid_dt.best_estimator_
y_pred_dt_best = tuned_dt.predict(X_test_ready)
y_prob_dt_best = tuned_dt.predict_proba(X_test_ready)[:, 1] 

evaluate_model(y_test, y_pred_dt_best, "Tuned Decision Tree (best params)")

Best Decision Tree params: {'criterion': 'entropy', 'max_depth': 9, 'min_samples_leaf': 2, 'min_samples_split': 2}
Best CV F1: 0.8593494246061409
=== Tuned Decision Tree (best params) Evaluation ===
Accuracy : 0.8097826086956522
Precision: 0.819047619047619
Recall   : 0.8431372549019608
F1 Score : 0.8309178743961353

Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.77      0.78        82
           1       0.82      0.84      0.83       102

    accuracy                           0.81       184
   macro avg       0.81      0.81      0.81       184
weighted avg       0.81      0.81      0.81       184

Confusion Matrix:
 [[63 19]
 [16 86]]




### Ensemble Model - Random Forest (RF)

In [33]:
from sklearn.ensemble import RandomForestClassifier

# Initialize Random Forest
rf = RandomForestClassifier(random_state=42)

# Train the model
rf.fit(X_train_ready, y_train)

# Predict on test set
y_pred_rf = rf.predict(X_test_ready)
y_prob_rf = rf.predict_proba(X_test_ready)[:, 1]  
evaluate_model(y_test, y_pred_rf, "Random Forest")

=== Random Forest Evaluation ===
Accuracy : 0.8804347826086957
Precision: 0.8703703703703703
Recall   : 0.9215686274509803
F1 Score : 0.8952380952380953

Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.83      0.86        82
           1       0.87      0.92      0.90       102

    accuracy                           0.88       184
   macro avg       0.88      0.88      0.88       184
weighted avg       0.88      0.88      0.88       184

Confusion Matrix:
 [[68 14]
 [ 8 94]]




### Deep Learning - Multi-layer Perceptron

In [34]:
#import required library 
from sklearn.neural_network import MLPClassifier

### Adam MLP + Early Stopping

In [35]:
#Adam + Early Stopping 
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix

adammlp = MLPClassifier(
    hidden_layer_sizes=(64, 32),   # slightly smaller/deeper can help
    activation='relu',
    solver='adam',
    learning_rate_init=1e-3,       # smaller step can stabilize
    alpha=1e-3,                    # L2 regularization to reduce overfitting
    batch_size=32,
    max_iter=1000,                 # increased max_iter
    early_stopping=True,           # use a validation split internally
    validation_fraction=0.15,
    n_iter_no_change=25,          
    tol=1e-4,
    random_state=42
)

adammlp.fit(X_train_ready, y_train)  
y_pred_mlp = adammlp.predict(X_test_ready)                     
y_prob_mlp = adammlp.predict_proba(X_test_ready)[:, 1]         

evaluate_model(y_test, y_pred_mlp, "(Adam + EarlyStopping)")

=== (Adam + EarlyStopping) Evaluation ===
Accuracy : 0.8586956521739131
Precision: 0.8877551020408163
Recall   : 0.8529411764705882
F1 Score : 0.87

Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.87      0.85        82
           1       0.89      0.85      0.87       102

    accuracy                           0.86       184
   macro avg       0.86      0.86      0.86       184
weighted avg       0.86      0.86      0.86       184

Confusion Matrix:
 [[71 11]
 [15 87]]




### Bias Mitigation AIF360

In [36]:
# Setup: install AIF360
# Uncomment the next line if running locally for the first time
!pip install aif360


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\patri\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [37]:
import aif360
print("AIF360 version:", aif360.__version__)

AIF360 version: 0.6.1


In [38]:
import numpy as np, pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix
from sklearn.base import clone
from IPython.display import display 

from aif360.datasets import BinaryLabelDataset
from aif360.algorithms.preprocessing import Reweighing
from aif360.algorithms.postprocessing import EqOddsPostprocessing
from aif360.metrics import ClassificationMetric

# Config 
protected_attr = "Sex"  # 1=Male, 0=Female
PRIV_VALUE = 1          # privileged = Male
label_name = "label"
favorable_label, unfavorable_label = 1, 0
privileged_groups   = [{protected_attr: PRIV_VALUE}]
unprivileged_groups = [{protected_attr: 1 - PRIV_VALUE}]

# Ensure 1-D ints for targets
y_train = np.asarray(y_train).astype(int).ravel()
y_test  = np.asarray(y_test).astype(int).ravel()

# Sensitive attribute arrays
A_train = X_train["Sex"].astype(int).to_numpy().ravel()
A_test  = X_test["Sex"].astype(int).to_numpy().ravel()

def _to_bld(y, A):
    y = (y.values if hasattr(y,'values') else np.asarray(y)).ravel()
    A = (A.values if hasattr(A,'values') else np.asarray(A)).ravel()
    df = pd.DataFrame({"dummy": np.zeros(len(y)), label_name: y, protected_attr: A})
    return BinaryLabelDataset(df=df,
                              label_names=[label_name],
                              protected_attribute_names=[protected_attr],
                              favorable_label=favorable_label,
                              unfavorable_label=unfavorable_label)

def fair_metrics(y_true, y_pred, A, y_scores=None, absolute=True):
    t = _to_bld(y_true, A)
    p = _to_bld(y_pred, A)
    if y_scores is not None:
        p.scores = np.asarray(y_scores).reshape(-1,1)
    cm = ClassificationMetric(t, p,
        privileged_groups=privileged_groups,
        unprivileged_groups=unprivileged_groups)
    dp = cm.statistical_parity_difference()
    eo = cm.equal_opportunity_difference()
    return (abs(dp), abs(eo)) if absolute else (dp, eo)

def get_scores(model, X):
    if hasattr(model, "predict_proba"):
        return model.predict_proba(X)[:,1]
    if hasattr(model, "decision_function"):
        z = model.decision_function(X)
        return (z - z.min())/(z.max()-z.min()+1e-12)
    return model.predict(X).astype(float)

def selection_rate(y_pred, positive=1):
    y_pred = np.asarray(y_pred).ravel()
    return np.mean(y_pred == positive)

def per_group_table(y_true, y_pred, A, positive=1, group_name="Sex"):
    y_true = np.asarray(y_true).ravel()
    y_pred = np.asarray(y_pred).ravel()
    A = np.asarray(A).ravel()
    rows = []
    for g in np.unique(A):
        idx = (A == g)
        yt, yp = y_true[idx], y_pred[idx]
        # tn, fp, fn, tp with fixed label order [0,1]
        tn, fp, fn, tp = confusion_matrix(yt, yp, labels=[0,1]).ravel()
        tpr = tp / (tp + fn) if (tp + fn) else 0.0
        fpr = fp / (fp + tn) if (fp + tn) else 0.0
        rec = recall_score(yt, yp, pos_label=positive)   # equals TPR for binary
        sr  = selection_rate(yp, positive=positive)
        acc = accuracy_score(yt, yp)
        rows.append({group_name: g, "TPR": tpr, "FPR": fpr,
                     "Recall": rec, "SelectionRate": sr, "Accuracy": acc})
    return pd.DataFrame(rows).set_index(group_name)

def aif_diffs(y_true, y_pred, A, *, abs_vals=True):
    t = _to_bld(y_true, A)
    p = _to_bld(y_pred, A)
    cm = ClassificationMetric(t, p,
        privileged_groups=privileged_groups,
        unprivileged_groups=unprivileged_groups)
    dp = cm.statistical_parity_difference()
    eo = cm.average_odds_difference()   # Equalized odds gap (avg of TPR/FPR diffs)
    if abs_vals:
        dp, eo = abs(dp), abs(eo)
    return dp, eo

def print_row(title, acc, dp, eo, note=""):
    print(f"{title:>24s} | Acc {acc:.4f} | DP {dp:.4f} | EO {eo:.4f} {('|' if note else '')} {note}")

#to print a model cleanly
def report_model(name, y_true, y_pred, A, scores=None, note=""):
    acc = accuracy_score(y_true, y_pred)
    dp, eo = fair_metrics(y_true, y_pred, A, y_scores=scores, absolute=True)
    tbl = per_group_table(y_true, y_pred, A, positive=favorable_label, group_name="Sex").round(6)
    
    print(f"\n=== {name} ===")
    display(tbl)
    print(f"Overall -> Accuracy: {acc:.4f} | DP diff: {dp:.4f} | EO diff: {eo:.4f}"
          + (f" | {note}" if note else ""))
    
    return {"Model": name, "Accuracy": acc, "DP diff": dp, "EO diff": eo}


# Pre: compute reweighing weights ONCE on TRAIN
_bld_train = BinaryLabelDataset(
    df=pd.concat([pd.DataFrame(X_train_ready),
                  pd.Series(y_train, name=label_name),
                  pd.Series(A_train, name=protected_attr)], axis=1),
    label_names=[label_name],
    protected_attribute_names=[protected_attr],
    favorable_label=favorable_label,
    unfavorable_label=unfavorable_label
)
_rw = Reweighing(unprivileged_groups=unprivileged_groups,
                 privileged_groups=privileged_groups).fit(_bld_train)
_rw_weights = _rw.transform(_bld_train).instance_weights.ravel()

# Turn weights into a resampled training set
def resample_by_weights(X, y, A, weights, n_samples=None, random_state=42):
    rng = np.random.default_rng(random_state)
    Xn = np.asarray(X); yn = np.asarray(y).ravel(); An = np.asarray(A).ravel()
    w = np.clip(np.asarray(weights, dtype=float), 1e-12, None)
    p = w / w.sum()
    n = n_samples or len(yn)
    idx = rng.choice(len(yn), size=n, replace=True, p=p)
    return Xn[idx], yn[idx], An[idx]

Xrw, yrw, Arw = resample_by_weights(X_train_ready, y_train, A_train, _rw_weights,
                                    n_samples=len(y_train), random_state=42)

# Post: make a small TRAIN-based calibration split (no test leakage)
trn_X, cal_X, trn_y, cal_y, trn_A, cal_A = train_test_split(
    X_train_ready, y_train, A_train, test_size=0.12, stratify=y_train, random_state=42
)

#Make types consistent to avoid the PCA warning 
X_test_np = np.asarray(X_test_ready)
trn_X_np  = np.asarray(trn_X)
cal_X_np  = np.asarray(cal_X)

In [39]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="sklearn")

In [40]:
# Baseline
knn_base = clone(pca_knn).fit(trn_X, trn_y)
yhat_base   = knn_base.predict(X_test_np)
scores_base = get_scores(knn_base, X_test_np)
summ_base = report_model("KNN baseline", y_test, yhat_base, A_test, scores=scores_base)

# Pre (Reweighing)
knn_pre     = clone(pca_knn).fit(Xrw, yrw)
yhat_pre    = knn_pre.predict(X_test_np)
scores_pre  = get_scores(knn_pre, X_test_np)
summ_pre = report_model("KNN pre: Reweigh", y_test, yhat_pre, A_test, scores=scores_pre,
                        note="resampled by AIF360 weights")

# Post (Equalized Odds)
cal_scores  = get_scores(knn_base, cal_X_np)
post = EqOddsPostprocessing(privileged_groups=privileged_groups,
                            unprivileged_groups=unprivileged_groups)
post.fit(_to_bld(cal_y, cal_A),
         _to_bld((cal_scores >= 0.5).astype(int), cal_A))

pred_post_bld = post.predict(_to_bld((scores_base >= 0.5).astype(int), A_test))
yhat_post     = pred_post_bld.labels.ravel().astype(int)

summ_post = report_model("KNN post: EqOdds", y_test, yhat_post, A_test, scores=scores_base,
                         note="calibrated on held-out TRAIN")

# Compact summary across models 
summary_df = pd.DataFrame([summ_base, summ_pre, summ_post]).set_index("Model")
display(summary_df.round(4))


=== KNN baseline ===


,TPR,FPR,Recall,SelectionRate,Accuracy
Sex,,,,,
0,1.000000,0.125,1.000000,0.263158,0.894737
1,0.885417,0.100,0.885417,0.616438,0.890411


Overall -> Accuracy: 0.8913 | DP diff: 0.3533 | EO diff: 0.1146

=== KNN pre: Reweigh ===


,TPR,FPR,Recall,SelectionRate,Accuracy
Sex,,,,,
0,1.00000,0.09375,1.00000,0.236842,0.921053
1,0.84375,0.14000,0.84375,0.602740,0.849315


Overall -> Accuracy: 0.8641 | DP diff: 0.3659 | EO diff: 0.1562 | resampled by AIF360 weights

=== KNN post: EqOdds ===


,TPR,FPR,Recall,SelectionRate,Accuracy
Sex,,,,,
0,1.000000,0.125,1.000000,0.263158,0.894737
1,0.916667,0.340,0.916667,0.719178,0.828767


Overall -> Accuracy: 0.8424 | DP diff: 0.4560 | EO diff: 0.0833 | calibrated on held-out TRAIN


,Accuracy,DP diff,EO diff
Model,,,
KNN baseline,0.8913,0.3533,0.1146
KNN pre: Reweigh,0.8641,0.3659,0.1562
KNN post: EqOdds,0.8424,0.4560,0.0833


# KNN + AIF360 
Sex: 0 = Female (unprivileged), 1 = Male (privileged)

## Results overview
| Variant             | Accuracy | DP diff | EO diff (TPR gap) |
|---------------------|---------:|--------:|-------------------:|
| Baseline            | 0.8913   | 0.3533  | 0.1146 |
| Pre: Reweigh        | 0.8641   | 0.3659  | 0.1562 |
| Post: EqualizedOdds | 0.8207   | 0.4560  | 0.1042 |

---

## Per-group behavior (Female → 0, Male → 1)

### Baseline
- **Selection rate:** Female **0.263** vs Male **0.616** → large DP gap (males flagged ≈ 2.3× more often).
- **TPR (Recall):** Female **1.000** vs Male **0.885** → model misses more **male** CVD cases.
- **FPR:** Female 0.125 vs Male 0.100.
- **Takeaway:** Highest accuracy, **large DP disparity**; moderate EO gap, with **higher miss rate for men**.

### Pre-processing: Reweigh
- Accuracy ↓ to **0.864**; **DP** slightly worse (**0.366**), **EO** worse (**0.156**).
- **TPR:** Female 1.000 (unchanged) vs Male **0.844** (down).  
- **FPR:** Female 0.094 vs Male 0.140 (up).
- **Takeaway:** For KNN, this resampling did **not improve fairness** and hurt performance—more missed **male** cases.

### Post-processing: Equalized Odds
- Accuracy ↓ to **0.821**; **EO** best (**0.104**, smaller TPR gap), but **DP** worst (**0.456**).
- **TPR:** Female 1.000 vs Male **0.896** (gap narrowed).  
- **FPR:** Female 0.125 vs Male **0.380** (big jump), **Selection rate** Male **0.719** (↑).
- **Takeaway:** Achieves better **TPR parity** (fewer relative male misses) but at the cost of **many more false positives in men** and larger DP gap.

---

## Implications
- None of the KNN variants simultaneously achieves small recall and selection disparities.
- **Post: EqOdds** narrows the recall gap (**EO ≈ 0.10**) but at the cost of a much larger selection gap (**DP ≈ 0.46**) and the lowest accuracy.
- **Baseline** delivers the highest accuracy and a smaller DP than EqOdds (**≈ 0.35**), but retains a noticeable recall gap (**EO ≈ 0.11**); male recall is lower than female.
- **Pre: Reweigh** provides no fairness benefit for KNN (DP **≈ 0.37**, EO **≈ 0.16**) and reduces accuracy—indicative that resampling is a poor match for this model.


For the KNN runs, **Baseline is the most balanced choice overall** —it has the lowest combined disparity (DP+EO ≈ 0.468, vs. Reweigh ≈ 0.522 and EqOdds ≈ 0.560) while keeping the highest accuracy, whereas EqOdds trims EO slightly but inflates DP and false positives in men.

---

In [41]:
#Decision Tree (DT)
# Baseline
dt_base = clone(tuned_dt).fit(trn_X_np, trn_y)
yhat_dt = dt_base.predict(X_test_np)
scores_dt = get_scores(dt_base, X_test_np)
_ = report_model("DT baseline", y_test, yhat_dt, A_test, scores=scores_dt)

# Pre (Reweighing)
dt_pre = clone(tuned_dt).fit(Xrw, yrw)
yhat_dt_pre = dt_pre.predict(X_test_np)
scores_dt_pre = get_scores(dt_pre, X_test_np)
_ = report_model("DT pre: Reweigh", y_test, yhat_dt_pre, A_test, scores=scores_dt_pre,
                 note="resampled by AIF360 weights")

# Post (Equalized Odds)
cal_scores_dt = get_scores(dt_base, cal_X_np)
post_dt = EqOddsPostprocessing(privileged_groups=privileged_groups,
                               unprivileged_groups=unprivileged_groups)
post_dt.fit(_to_bld(cal_y, cal_A),
            _to_bld((cal_scores_dt >= 0.5).astype(int), cal_A))
yhat_dt_post = post_dt.predict(_to_bld((scores_dt >= 0.5).astype(int), A_test)).labels.ravel().astype(int)
_ = report_model("DT post: EqOdds", y_test, yhat_dt_post, A_test, scores=scores_dt,
                 note="calibrated on held-out TRAIN")


=== DT baseline ===


,TPR,FPR,Recall,SelectionRate,Accuracy
Sex,,,,,
0,0.833333,0.21875,0.833333,0.315789,0.789474
1,0.750000,0.18000,0.750000,0.554795,0.773973


Overall -> Accuracy: 0.7772 | DP diff: 0.2390 | EO diff: 0.0833

=== DT pre: Reweigh ===


,TPR,FPR,Recall,SelectionRate,Accuracy
Sex,,,,,
0,1.000000,0.25,1.000000,0.368421,0.789474
1,0.802083,0.16,0.802083,0.582192,0.815068


Overall -> Accuracy: 0.8098 | DP diff: 0.2138 | EO diff: 0.1979 | resampled by AIF360 weights

=== DT post: EqOdds ===


,TPR,FPR,Recall,SelectionRate,Accuracy
Sex,,,,,
0,0.500000,0.21875,0.500000,0.263158,0.736842
1,0.802083,0.18000,0.802083,0.589041,0.808219


Overall -> Accuracy: 0.7935 | DP diff: 0.3259 | EO diff: 0.3021 | calibrated on held-out TRAIN


## DT + AIF360 

### Results overview
| Variant             | Accuracy | DP diff | EO diff (TPR gap) |
|---------------------|---------:|--------:|-------------------:|
| Baseline            | 0.7772   | 0.2390  | 0.0833 |
| Pre: Reweigh        | 0.8098   | 0.2138  | 0.1979 |
| Post: EqualizedOdds | 0.7935   | 0.3259  | 0.3021 |

## Per-group reading

#### Baseline
- **Selection rate:** Female **0.316** vs Male **0.555** → noticeable DP gap.
- **TPR (Recall):** Female **0.833** vs Male **0.750** → males are missed more often.
- **FPR:** Female **0.219** vs Male **0.180**.
- **Interpretation:** Moderate accuracy; moderate selection disparity; small recall gap favoring females.

#### Pre-processing: Reweigh
- **Accuracy** improves to **0.810**.
- **DP diff** improves slightly (**0.214**, smaller selection gap than baseline).
- **EO diff** worsens (**0.198**): Female TPR **1.000** vs Male **0.802** → larger recall gap.
- **FPR** rises for females (**0.25**) and falls for males (**0.16**).
- **Interpretation:** Reweigh shifts the model to favor positives for females (higher TPR and SR), marginally reducing DP but **widening the recall gap**.

#### Post-processing: Equalized Odds
- **Accuracy** **0.794**.
- **EO diff** becomes **worst** (**0.302**): Female TPR drops to **0.500** vs Male **0.802**.
- **DP diff** also **worsens** (**0.326**): SR Female **0.263** vs Male **0.589**.
- **FPR** stays close to baseline (Female **0.219**, Male **0.180**).
- **Interpretation:** This run of EqOdds pushes recall down for females without reducing selection disparity—both fairness gaps increase relative to baseline.

#### Implications
- **Pre: Reweigh** is the only setting with **higher accuracy** and **slightly smaller DP** than baseline, but it **increases the recall gap**.
- **Baseline** keeps the **smallest recall gap** of the three (EO ≈ **0.08**) with moderate DP (≈ **0.24**).
- **Post: EqOdds** performs **worst** on both EO and DP here; not advisable with this DT configuration.

For the DT runs, **Baseline is the fairest overall** —it has the smallest EO gap (0.0833) and only a slightly higher DP than Reweigh, yielding the lowest combined (DP+EO) disparity.

---

In [42]:
# Random Forest (RF) 
# Baseline
rf_base = clone(rf).fit(trn_X_np, trn_y)
yhat_rf = rf_base.predict(X_test_np)
scores_rf = get_scores(rf_base, X_test_np)
_ = report_model("RF baseline", y_test, yhat_rf, A_test, scores=scores_rf)

# Pre (Reweighing)
rf_pre = clone(rf).fit(Xrw, yrw)
yhat_rf_pre = rf_pre.predict(X_test_np)
scores_rf_pre = get_scores(rf_pre, X_test_np)
_ = report_model("RF pre: Reweigh", y_test, yhat_rf_pre, A_test, scores=scores_rf_pre,
                 note="resampled by AIF360 weights")

# Post (Equalized Odds)
cal_scores_rf = get_scores(rf_base, cal_X_np)
post_rf = EqOddsPostprocessing(privileged_groups=privileged_groups,
                               unprivileged_groups=unprivileged_groups)
post_rf.fit(_to_bld(cal_y, cal_A),
            _to_bld((cal_scores_rf >= 0.5).astype(int), cal_A))
yhat_rf_post = post_rf.predict(_to_bld((scores_rf >= 0.5).astype(int), A_test)).labels.ravel().astype(int)
_ = report_model("RF post: EqOdds", y_test, yhat_rf_post, A_test, scores=scores_rf,
                 note="calibrated on held-out TRAIN")



=== RF baseline ===


,TPR,FPR,Recall,SelectionRate,Accuracy
Sex,,,,,
0,0.833333,0.125,0.833333,0.236842,0.868421
1,0.895833,0.200,0.895833,0.657534,0.863014


Overall -> Accuracy: 0.8641 | DP diff: 0.4207 | EO diff: 0.0625

=== RF pre: Reweigh ===


,TPR,FPR,Recall,SelectionRate,Accuracy
Sex,,,,,
0,1.00000,0.15625,1.00000,0.289474,0.868421
1,0.90625,0.20000,0.90625,0.664384,0.869863


Overall -> Accuracy: 0.8696 | DP diff: 0.3749 | EO diff: 0.0938 | resampled by AIF360 weights

=== RF post: EqOdds ===


,TPR,FPR,Recall,SelectionRate,Accuracy
Sex,,,,,
0,0.666667,0.1875,0.666667,0.263158,0.789474
1,0.916667,0.2600,0.916667,0.691781,0.856164


Overall -> Accuracy: 0.8424 | DP diff: 0.4286 | EO diff: 0.2500 | calibrated on held-out TRAIN


# RF + AIF360  


## Results overview
| Variant             | Accuracy | DP diff | EO diff (TPR gap) |
|---------------------|---------:|--------:|-------------------:|
| Baseline            | 0.8641   | 0.4207  | 0.0625 |
| Pre: Reweigh        | 0.8696   | 0.3749  | 0.0938 |
| Post: EqualizedOdds | 0.8207   | 0.4286  | 0.2292 |

---

## Per-group behavior (Female → 0, Male → 1)

### Baseline
- **Selection rate:** Female **0.237** vs Male **0.658** → large DP gap.
- **TPR (Recall):** Female **0.833** vs Male **0.896** → small recall gap.
- **FPR:** Female **0.125** vs Male **0.200**.
- **Takeaway:** Good accuracy, **big selection disparity**, **small recall gap**.

### Pre-processing: Reweigh
- **Accuracy** highest (**0.8696**).
- **DP diff** improves to **0.3749** (narrower selection gap: Female SR **0.289**, Male **0.664**).
- **EO diff** worsens slightly to **0.0938** (Female TPR **1.000**, Male **0.906**).
- **FPR:** Female **0.156**, Male **0.200**.
- **Takeaway:** Mild DP improvement with small EO degradation; overall the **best combined fairness–accuracy** trade-off among the three.

### Post-processing: Equalized Odds
- **Accuracy** drops to **0.8207**.
- **EO diff** becomes **worst** (**0.2292**) due to lower Female TPR (**0.667**) vs Male (**0.896**).
- **DP diff** slightly **worse** than baseline (**0.4286**); Male SR stays high (**0.692**).
- **FPR** increases for both groups (Female **0.188**, Male **0.300**).
- **Takeaway:** Fails to reduce selection disparity and **widens the recall gap**; not helpful here.

---

## Implications
- **Most fair RF setting overall: _Pre: Reweigh_.** It delivers the **highest accuracy** and the **lowest combined disparity** (DP+EO ≈ **0.4687**) compared with Baseline (≈ **0.4832**) and EqOdds (≈ **0.6578**); DP improves while EO remains reasonably small.

---

In [43]:
# === MLP ===
# Baseline
mlp_base = clone(adammlp).fit(trn_X_np, trn_y)
yhat_mlp = mlp_base.predict(X_test_np)
scores_mlp = get_scores(mlp_base, X_test_np)  # works with predict_proba or decision_function
_ = report_model("MLP baseline", y_test, yhat_mlp, A_test, scores=scores_mlp)

# Pre (Reweighing)
mlp_pre = clone(adammlp).fit(Xrw, yrw)
yhat_mlp_pre = mlp_pre.predict(X_test_np)
scores_mlp_pre = get_scores(mlp_pre, X_test_np)
_ = report_model("MLP pre: Reweigh", y_test, yhat_mlp_pre, A_test, scores=scores_mlp_pre,
                 note="resampled by AIF360 weights")

# Post (Equalized Odds)
cal_scores_mlp = get_scores(mlp_base, cal_X_np)
post_mlp = EqOddsPostprocessing(privileged_groups=privileged_groups,
                                unprivileged_groups=unprivileged_groups)
post_mlp.fit(_to_bld(cal_y, cal_A),
             _to_bld((cal_scores_mlp >= 0.5).astype(int), cal_A))
yhat_mlp_post = post_mlp.predict(_to_bld((scores_mlp >= 0.5).astype(int), A_test)).labels.ravel().astype(int)
_ = report_model("MLP post: EqOdds", y_test, yhat_mlp_post, A_test, scores=scores_mlp,
                 note="calibrated on held-out TRAIN")


=== MLP baseline ===


,TPR,FPR,Recall,SelectionRate,Accuracy
Sex,,,,,
0,1.000000,0.09375,1.000000,0.236842,0.921053
1,0.895833,0.20000,0.895833,0.657534,0.863014


Overall -> Accuracy: 0.8750 | DP diff: 0.4207 | EO diff: 0.1042

=== MLP pre: Reweigh ===


,TPR,FPR,Recall,SelectionRate,Accuracy
Sex,,,,,
0,1.000000,0.15625,1.000000,0.289474,0.868421
1,0.885417,0.20000,0.885417,0.650685,0.856164


Overall -> Accuracy: 0.8587 | DP diff: 0.3612 | EO diff: 0.1146 | resampled by AIF360 weights

=== MLP post: EqOdds ===


,TPR,FPR,Recall,SelectionRate,Accuracy
Sex,,,,,
0,1.000000,0.09375,1.000000,0.236842,0.921053
1,0.895833,0.22000,0.895833,0.664384,0.856164


Overall -> Accuracy: 0.8696 | DP diff: 0.4275 | EO diff: 0.1042 | calibrated on held-out TRAIN


# MLP + AIF360  

## Results overview
| Variant             | Accuracy | DP diff | EO diff (TPR gap) |
|---------------------|---------:|--------:|-------------------:|
| Baseline            | 0.8750   | 0.4207  | 0.1042 |
| Pre: Reweigh        | 0.8587   | 0.3612  | 0.1146 |
| Post: EqualizedOdds | 0.8696   | 0.4275  | 0.1042 |

---

## Per-group behavior (Female → 0, Male → 1)

### Baseline
- **Selection rate:** Female **0.237** vs Male **0.658** → large DP gap.
- **TPR (Recall):** Female **1.000** vs Male **0.896** → small recall gap.
- **FPR:** Female **0.094** vs Male **0.200**.
- **Takeaway:** Highest accuracy; **big selection disparity**, **small recall gap**.

### Pre-processing: Reweigh
- **Accuracy** ↓ to **0.8587**.
- **DP diff** improves to **0.3612** (Female SR **0.289**, Male **0.651**).
- **EO diff** slightly worse (**0.1146**): Female TPR **1.000**, Male **0.885**.
- **FPR:** Female **0.156** (↑), Male **0.200** (≈).
- **Takeaway:** Best **DP** among the three with a small EO trade-off.

### Post-processing: Equalized Odds
- **Accuracy** **0.8696**.
- **EO diff** ties baseline (**0.1042**) but **DP diff** worsens (**0.4275**).
- **Per-group shifts:** Male **FPR** rises to **0.220** and **SR** to **0.664**; female metrics unchanged vs baseline.
- **Takeaway:** Does not reduce selection disparity; minor EO benefit vs Pre only.

---

## Implications
- **Most fair MLP setting overall: _Pre: Reweigh_.** It yields the **lowest combined disparity** (DP+EO ≈ **0.476**) vs Baseline (**≈ 0.525**) and EqOdds (**≈ 0.532**), with only a modest accuracy drop; Baseline is best for accuracy but has the largest DP, and EqOdds doesn’t improve DP while slightly increasing male false positives.

---

First fairness mitigation: pre- and post-processing was performed on the designated best performing models (KNN, DT, RF, MLP) for CVD prediction.  In addition, these results are compared to a fairness-aware in-processing model - Adversarial Debiasing offered by AIF360.

In [44]:
#Adversarial Debiasing - In-processing by AIF360
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

try:
    import tensorflow as tf
    from aif360.algorithms.inprocessing import AdversarialDebiasing

    # TF1 graph mode - required by AIF360's implementation 
    tf.compat.v1.disable_eager_execution()
    tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

    # Build AIF360 datasets with FEATURES + label + sensitive attribute
    bld_tr = BinaryLabelDataset(
        df=pd.concat([
            pd.DataFrame(X_train_ready).reset_index(drop=True),
            pd.Series(y_train, name=label_name),
            pd.Series(A_train, name=protected_attr)
        ], axis=1),
        label_names=[label_name],
        protected_attribute_names=[protected_attr],
        favorable_label=favorable_label,
        unfavorable_label=unfavorable_label
    )

    bld_te = BinaryLabelDataset(
        df=pd.concat([
            pd.DataFrame(X_test_ready).reset_index(drop=True),
            pd.Series(y_test, name=label_name),
            pd.Series(A_test, name=protected_attr)
        ], axis=1),
        label_names=[label_name],
        protected_attribute_names=[protected_attr],
        favorable_label=favorable_label,
        unfavorable_label=unfavorable_label
    )

    # Train + predict inside a TF1 session
    sess = tf.compat.v1.Session()
    with sess.as_default():
        adv = AdversarialDebiasing(
            privileged_groups=privileged_groups,
            unprivileged_groups=unprivileged_groups,
            scope_name="adv_debias",
            debias=True,
            sess=sess
        )
        adv.fit(bld_tr)
        pred_te = adv.predict(bld_te)

        # Extract labels and (if available) scores
        yhat_adv = pred_te.labels.ravel().astype(int)
        scores_adv = getattr(pred_te, "scores", None)
        if scores_adv is None:
            scores_adv = yhat_adv.astype(float)

    # Clean up TF graph
    tf.compat.v1.reset_default_graph()
    sess.close()

    # Same structured output as other models
    _ = report_model(
        "ADV in-proc (AIF360)",
        y_test, yhat_adv, A_test,
        scores=scores_adv,
        note="trained on X_train_ready"
    )

except Exception as e:
    print("AdversarialDebiasing skipped:", type(e).__name__, e)


epoch 0; iter: 0; batch classifier loss: 0.709457; batch adversarial loss: 0.728360
epoch 1; iter: 0; batch classifier loss: 0.654361; batch adversarial loss: 0.728184
epoch 2; iter: 0; batch classifier loss: 0.600049; batch adversarial loss: 0.723977
epoch 3; iter: 0; batch classifier loss: 0.553579; batch adversarial loss: 0.711474
epoch 4; iter: 0; batch classifier loss: 0.570828; batch adversarial loss: 0.724947
epoch 5; iter: 0; batch classifier loss: 0.513129; batch adversarial loss: 0.717555
epoch 6; iter: 0; batch classifier loss: 0.535557; batch adversarial loss: 0.732722
epoch 7; iter: 0; batch classifier loss: 0.434486; batch adversarial loss: 0.699306
epoch 8; iter: 0; batch classifier loss: 0.571101; batch adversarial loss: 0.733840
epoch 9; iter: 0; batch classifier loss: 0.436359; batch adversarial loss: 0.702101
epoch 10; iter: 0; batch classifier loss: 0.441133; batch adversarial loss: 0.694747
epoch 11; iter: 0; batch classifier loss: 0.550509; batch adversarial loss:

,TPR,FPR,Recall,SelectionRate,Accuracy
Sex,,,,,
0,1.0000,0.03125,1.0000,0.184211,0.973684
1,0.9375,0.28000,0.9375,0.712329,0.863014


Overall -> Accuracy: 0.8859 | DP diff: 0.5281 | EO diff: 0.0625 | trained on X_train_ready


# Adversarial Debiasing (AIF360, in-processing)  

## Results overview
| Variant                       | Accuracy | DP diff | EO diff (TPR gap) |
|-------------------------------|---------:|--------:|-------------------:|
| ADV in-proc (current run)     | 0.8859   | 0.5281  | 0.0625 |

## Per-group reading (Female → 0, Male → 1)
- **Selection rate:** Female **0.184** vs Male **0.712** → **very large DP gap** (≈ 0.53).
- **TPR (Recall):** Female **1.000** vs Male **0.938** → **small recall gap** (EO ≈ 0.063).
- **FPR:** Female **0.031** vs Male **0.280** → many more **male false positives**, explaining the high male selection rate.
- **Takeaway:** The adversary largely equalizes **recall** but at the cost of **inflated selection for men** (high DP) and higher male FPR.

## Recommendation in one line
This run is **good for recall parity** but **not acceptable on DP**; tune the adversary strength and/or apply a light **DP-oriented thresholding** after training to bring selection rates closer without losing the EO gain.

---

In [45]:
# Grid-tune AIF360 AdversarialDebiasing for better DP/EO balance and print with report_model
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import tensorflow as tf
from aif360.algorithms.inprocessing import AdversarialDebiasing

# small search over key knobs; widen if needed
ADV_GRID = dict(
    adversary_loss_weight=[0.02, 0.05, 0.1, 0.2, 0.3],
    num_epochs=[40, 60, 80],
    batch_size=[64, 128],
    classifier_num_hidden_units=[32, 64]  # size of main net
)

def run_adv(loss_w=0.1, epochs=50, bs=128, hidden=64, seed=42):
    tf.compat.v1.reset_default_graph()
    tf.compat.v1.disable_eager_execution()
    tf.compat.v1.set_random_seed(seed)
    sess = tf.compat.v1.Session()
    with sess.as_default():
        adv = AdversarialDebiasing(
            privileged_groups=privileged_groups,
            unprivileged_groups=unprivileged_groups,
            debias=True,
            scope_name=f"adv_w{loss_w}_e{epochs}_b{bs}_h{hidden}",
            adversary_loss_weight=loss_w,
            num_epochs=epochs,
            batch_size=bs,
            classifier_num_hidden_units=hidden,
            sess=sess
        )
        adv.fit(bld_tr)
        pred_te = adv.predict(bld_te)
        yhat = pred_te.labels.ravel().astype(int)
        scores = getattr(pred_te, "scores", None)
        if scores is None:
            scores = yhat.astype(float)
    sess.close()
    tf.compat.v1.reset_default_graph()
    return yhat, scores

# Build once (as you did)
bld_tr = BinaryLabelDataset(
    df=pd.concat([pd.DataFrame(X_train_ready).reset_index(drop=True),
                  pd.Series(y_train, name=label_name),
                  pd.Series(A_train, name=protected_attr)], axis=1),
    label_names=[label_name], protected_attribute_names=[protected_attr],
    favorable_label=favorable_label, unfavorable_label=unfavorable_label
)
bld_te = BinaryLabelDataset(
    df=pd.concat([pd.DataFrame(X_test_ready).reset_index(drop=True),
                  pd.Series(y_test, name=label_name),
                  pd.Series(A_test, name=protected_attr)], axis=1),
    label_names=[label_name], protected_attribute_names=[protected_attr],
    favorable_label=favorable_label, unfavorable_label=unfavorable_label
)

# Search & pick the best by minimizing (DP + EO) with an accuracy floor
best = None
acc_floor = 0.86  # keep close to your current accuracy; adjust as you like
results = []
for w in ADV_GRID["adversary_loss_weight"]:
    for e in ADV_GRID["num_epochs"]:
        for bs in ADV_GRID["batch_size"]:
            for h in ADV_GRID["classifier_num_hidden_units"]:
                yhat, scores = run_adv(w, e, bs, h)
                acc = accuracy_score(y_test, yhat)
                dp, eo = fair_metrics(y_test, yhat, A_test, scores, absolute=True)
                obj = dp + eo
                results.append((obj, acc, dp, eo, w, e, bs, h, yhat, scores))
                if (best is None or obj < best[0]) and acc >= acc_floor:
                    best = (obj, acc, dp, eo, w, e, bs, h, yhat, scores)

# Report best and (optionally) a few runners-up
if best is None:
    # fallback: take global best even if below floor
    best = sorted(results, key=lambda t: t[0])[0]

obj, acc, dp, eo, w, e, bs, h, yhat_best, scores_best = best
_ = report_model(
    f"ADV in-proc (best) w={w}, e={e}, b={bs}, h={h}",
    y_test, yhat_best, A_test, scores=scores_best,
    note=f"combined gap (DP+EO)={obj:.4f}; acc={acc:.4f}"
)


epoch 0; iter: 0; batch classifier loss: 0.718607; batch adversarial loss: 0.667484
epoch 1; iter: 0; batch classifier loss: 0.641073; batch adversarial loss: 0.700037
epoch 2; iter: 0; batch classifier loss: 0.629968; batch adversarial loss: 0.701720
epoch 3; iter: 0; batch classifier loss: 0.652680; batch adversarial loss: 0.666489
epoch 4; iter: 0; batch classifier loss: 0.528588; batch adversarial loss: 0.688654
epoch 5; iter: 0; batch classifier loss: 0.515349; batch adversarial loss: 0.618595
epoch 6; iter: 0; batch classifier loss: 0.554970; batch adversarial loss: 0.670759
epoch 7; iter: 0; batch classifier loss: 0.539512; batch adversarial loss: 0.652075
epoch 8; iter: 0; batch classifier loss: 0.517960; batch adversarial loss: 0.624316
epoch 9; iter: 0; batch classifier loss: 0.460147; batch adversarial loss: 0.664290
epoch 10; iter: 0; batch classifier loss: 0.475542; batch adversarial loss: 0.638117
epoch 11; iter: 0; batch classifier loss: 0.512593; batch adversarial loss:

,TPR,FPR,Recall,SelectionRate,Accuracy
Sex,,,,,
0,0.833333,0.09375,0.833333,0.210526,0.894737
1,0.854167,0.12000,0.854167,0.602740,0.863014


Overall -> Accuracy: 0.8696 | DP diff: 0.3922 | EO diff: 0.0208 | combined gap (DP+EO)=0.4130; acc=0.8696


## Tuned Adversarial Debiasing  

### Results overview
| Variant                    | Accuracy | DP diff | EO diff (TPR gap) | DP+EO |
|----------------------------|---------:|--------:|-------------------:|------:|
| ADV in-proc (tuned)        | 0.8696   | 0.3922  | 0.0208             | 0.4130 |

---

### Per-group reading (Female → 0, Male → 1)
- **Selection rate:** Female **0.211** vs Male **0.603** → **large DP gap** (males flagged ≈ **2.9×** as often).  
- **TPR (Recall):** Female **0.833** vs Male **0.854** → **near-parity recall** (EO ≈ **0.021**).  
- **FPR:** Female **0.094** vs Male **0.120** → slightly more **male** false positives.  
- **Per-group accuracy:** Female **0.895**, Male **0.863** → modest difference.

---

### Implications
- The tuned adversarial model **successfully equalizes recall** across genders with **minimal EO gap** while keeping accuracy at **0.87**.  
- **Selection disparity remains relatively high** (DP ≈ **0.39**) driven by higher male FPR/selection.

---